# A Smarter Implementation

We should be able to search smarter.

In [9]:
import random
import logging
from typing import Generator
from math import comb, log, ceil, floor


logger = logging.getLogger()
logger.handlers.clear()
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)


def find_min_value_given_probability(N, n, target_probability):
    # Calculate the total combinations for the sample
    total_combinations = comb(N, n)

    logger.debug(f'N = {N}')
    logger.debug(f'n = {n}')
    logger.debug(f'target_probability = {target_probability}')
    
    m = 1
    # Smallest known m where P(m) > target.
    m_min = m - 1
    # Largest known m where P(m) <= target.
    m_max = N - n + 1
    
    while m <= m_max:
        current_combination = comb(N - m, n)
        probability = current_combination / total_combinations

        logger.debug(f'm = {m}')
        logger.debug(f'm_min = {m_min}')
        logger.debug(f'm_max = {m_max}')
        logger.debug(f'probability = {probability}')
        
        if probability <= target_probability:
            if m_min + 1 == m:
                return m
            else:
                m_max = m
                m -= ceil((m - m_min) / 2)
        else:
            m_min = m
            m_step = ceil(log(target_probability) / log(probability))
            logger.debug(f'm_step = {m_step}')
            m_max = min(m_max, m + m_step)
            logger.debug(f'next m_max = {m_max}')
            m += ceil((m_max - m) / 2)

    return m


def generate_unique_random_numbers(min_num: int, max_num: int, batch_size: int) -> Generator[int, None, None]:
    if max_num - min_num + 1 < batch_size:
        raise ValueError("Range is too small to generate the requested number of unique random numbers.")

    n = batch_size

    while n > 0:
        N = max_num - min_num + 1
        random_probability = random.random()
        logger.debug(f'min_num = {min_num}')
        logger.debug(f'max_num = {max_num}')
        m = find_min_value_given_probability(N, n, random_probability)
        logger.debug(f'result m = {m}')
        number = min_num + m - 1
        logger.debug(f'number = {number}')
        logger.debug('')
        min_num = number + 1
        n -= 1
        yield number

In [7]:
list(generate_unique_random_numbers(1, 10, 3))

[1, 3, 8]

## Distribution

The distribution of this approach is even.

In [ ]:
import run

logger.setLevel(logging.DEBUG)

run.plot(generate_unique_random_numbers, 0, 9, 3, 1000)

## Execution Time

The execution time here is terrible. Can we do better?

In [ ]:
import plot

df = run.measure_time(lambda range_size, batch_size: generate_unique_random_numbers(1, range_size, batch_size), range(100,100))
plot.execution_time(df)

## Memory Usage

The memory usage is constant.